Verschiedene Abrufmethoden für die Dokumente mit dem Modell: infloat/multilingual-e5-large

Der SelfQueryRetriever gibt keinen Score aus.

- Retrieval ohne Filter
- Retrieval mit optimalen, selbst gesetzten Filtern
- Retrieval mit SelfQueryRetriever und den generierten Filtern

Da es sich um 4 Collections handelt, sind es 12 verschiedene Abrufe.

Die Textdateien, die hier erstellt werden, dienen nur zum Nachvollziehen, der abgerufenen Dokumente. Der Score ist an der Stelle nicht wichtig.

Auch hier wurde mit den umformulierten Fragen gearbeitet, um nachzuvollziehen, wie sich der SelfQueryRetriever verhält.

Erst im Verlauf der Evaluation mit den ursprünglichen und umformulierten Fragen hat sich herausgestellt, dass der SelfQueryRetriever nicht mit den anderen Methoden verglichen werden kann.

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import RetrievalMode, QdrantVectorStore
from qdrant_client.models import Filter, FieldCondition, MatchValue
import os
import re
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Verhindert inkonsistenten Fehler beim Erzeugen von Embeddings (sonst wurde versucht parallel auf die gleiche Ressource zuzugreifen)
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
# Query Präfix für Embeddings der Fragen
encode_kwargs = {'prompt': 'query: '}

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large", encode_kwargs=encode_kwargs)

/Users/bastian/Desktop/BachelorProjektAbgabe/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:datasets:PyTorch version 2.6.0 available.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-large


In [4]:
# Qdrant-Client für Token-basierte-Collection
token_collection = "intfloat_multilingual-e5-large_token_based_chunks"
qdrant_url = "http://localhost:6333"
token_client = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    prefer_grpc=False,
    collection_name=token_collection,
    url=qdrant_url,
)

INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks "HTTP/1.1 200 OK"


In [5]:
# Qdrant-Client für Recursive-Collection
recursive_collection = "intfloat_multilingual-e5-large_recursive_chunks"
recursive_client = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    prefer_grpc=False,
    collection_name=recursive_collection,
    url=qdrant_url,
)

INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks "HTTP/1.1 200 OK"


In [6]:
fragen = {
    1: "Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben?",
    7: "Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen?",
    10: "Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen?"
}

In [7]:
test_result = token_client.similarity_search_with_score(fragen[10], k=10)

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


In [8]:
print(test_result)

[(Document(metadata={'id': 1, 'studiengang': 'Wirtschaftsinformatik', 'abschluss': 'Bachelor', 'standort': 'Hagen', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/wirtschaftsinformatik__b_sc__/wirtschaftsinformatik__b_sc__1.php', 'hash': '52d4e379d2abc0535448fc42b8962da89d850e7865654a3ea8a44648e2549a4a', '_id': '2e1ed528-b0aa-4396-8531-1910430076c4', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='# Wirtschaftsinformatik B.Sc. ## Hagen 1. Studienangebot 2. Studiengänge 3. Hagen 4. Wirtschaftsinformatik (B.Sc.) ## Bachelor-Studium Wirtschaftsinformatik ### Interdisziplinarität! Das ist der große Trumpf des Bachelor-Studiengangs Wirtschaftsinformatik. Er kombiniert Inhalte aus den Bereichen Informatik, Betriebswirtschaftslehre (BWL) und Technik zu einer attraktiven Mischung mit besten beruflichen Perspektiven. Auf dem Lehrplan stehen Fächer wie IT-Sicherheit, Business Intelligence oder Web-Programmierung. Dazu gibt es eine fundierte

In [9]:
# Fragen mit Dokumenten und Relevanz in Textdateien speichern, damit man es dauerhaft nachvollziehen kann:
output_dir = "1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz"
os.makedirs(output_dir, exist_ok=True)

In [10]:
def clean_filename(text: str) -> str:
    text = re.sub(r'[<>:"/\\|?*]', '', text)  # Regex-Pattern, um Fragen als Dateinamen zu nutzen
    return text.strip()

In [11]:
for id, frage in fragen.items():
    frage_filename = f"{id}_{clean_filename(frage)}.txt"  # Dateiname mit ID ergänzen
    filepath = os.path.join(output_dir, frage_filename)

    found_docs = token_client.similarity_search_with_score(frage, k=10)

    with open(filepath, "w", encoding="utf-8") as file:
        file.write(f"ID: {id}\n")
        file.write(f"Frage: {frage}\n\n")

        # Rückgaben formatiert mit Metadaten in Textdateien speichern
        for idx, (doc, score) in enumerate(found_docs, 1):
            file.write(f"Dokument {idx} (Score: {score:.8f}):\n")

            file.write("Metadaten:\n")
            for key, value in doc.metadata.items():
                file.write(f"  - {key}: {value}\n")

            file.write("\nInhalt:\n")
            file.write(f"{doc.page_content}\n\n")

    print(f"Gespeichert: {filepath}")

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt
Gespeichert: 1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt
Gespeichert: 1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz/3_Welche Fachgebiete sind an der FH vertreten.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz/4_Ist eine Beurlaubung während des Studiums möglich.txt
Gespeichert: 1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz/5_Welche Studienmodelle werden angeboten.txt
Gespeichert: 1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz/6_Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben.txt
Gespeichert: 1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz/7_Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt
Gespeichert: 1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt
Gespeichert: 1-infloat_token_based-Fragen_mit_Dokumenten_und_Relevanz/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt


In [12]:
output_dir = "2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz"
os.makedirs(output_dir, exist_ok=True)

In [13]:
for id, frage in fragen.items():
    frage_filename = f"{id}_{clean_filename(frage)}.txt"  # Dateiname mit ID ergänzen
    filepath = os.path.join(output_dir, frage_filename)

    found_docs = recursive_client.similarity_search_with_score(frage, k=10)

    with open(filepath, "w", encoding="utf-8") as file:
        file.write(f"ID: {id}\n")
        file.write(f"Frage: {frage}\n\n")

        for idx, (doc, score) in enumerate(found_docs, 1):
            file.write(f"Dokument {idx} (Score: {score:.8f}):\n")

            file.write("Metadaten:\n")
            for key, value in doc.metadata.items():
                file.write(f"  - {key}: {value}\n")

            file.write("\nInhalt:\n")
            file.write(f"{doc.page_content}\n\n")

    print(f"Gespeichert: {filepath}")

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt
Gespeichert: 2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt
Gespeichert: 2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz/3_Welche Fachgebiete sind an der FH vertreten.txt
Gespeichert: 2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz/4_Ist eine Beurlaubung während des Studiums möglich.txt
Gespeichert: 2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz/5_Welche Studienmodelle werden angeboten.txt
Gespeichert: 2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz/6_Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz/7_Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut.txt
Gespeichert: 2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt
Gespeichert: 2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt
Gespeichert: 2-infloat_recursive-Fragen_mit_Dokumenten_und_Relevanz/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt


# Fragen mit Filtern abrufen


In [ ]:
def create_field_condition(field, value):
    return FieldCondition(
        key=f"metadata.{field}",
        match=MatchValue(value=value)
    )

'''
Das hier sind die idealen Filter für die jeweilige Frage.

Bei Fragen zu allgemeinen Informationen, die nicht auf einen spezifischen Standort, Studiengang oder Abschluss abzielen, muss 'Alle' als Wert für den Filter verwendet werden.

Bei allgemeinen Fragen zu einem Standort muss der Standort als Wert für den Filter verwendet werden. Der Wert für Studiengang und Abschluss benötigt den Wert Alle.
(Diese Daten wurden später aufgrund ihres geringen Umfangs nicht mehr in der Evaluation betrachtet und sollten daher auch ignoriert werden.)


In der Arbeit geht es darum, ob das Filtern überhaupt eine positive Auswirkung auf das Retrieval hat.
Sollte dies nicht der Fall sein, könnte darauf verzichtet werden die Filter zu nutzen. Sonst muss ein Lösungsansatz im GUI einer späteren App mit Drop-Down-Menüs implementiert werden oder der SelfQueryRetriever genutzt werden.
'''
filters = {
    1: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Meschede"),
        create_field_condition("abschluss", "Master")
    ]),
    2: Filter(must=[
        create_field_condition("standort", "Soest"),
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    3: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    4: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    5: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    6: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    7: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Soest"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    8: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    9: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    10: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
}


In [15]:
# Fragen mit Filtern abrufen
output_dir = "3-infloat_token_based-Fragen_mit_Filtern"
os.makedirs(output_dir, exist_ok=True)

for id, frage in fragen.items():
    frage_filename = f"{id}_{clean_filename(frage)}.txt"  # Dateiname mit ID ergänzen
    filepath = os.path.join(output_dir, frage_filename)

    logging.info(f"Nutze für Frage Nr. {id}. {frage} den Filter: {filters[id]}")
    found_docs = token_client.similarity_search_with_score(frage, k=10, filter=filters[id])

    with open(filepath, "w", encoding="utf-8") as file:
        file.write(f"ID: {id}\n")
        file.write(f"Frage: {frage}\n\n")

        for idx, (doc, score) in enumerate(found_docs, 1):
            file.write(f"Dokument {idx} (Score: {score:.8f}):\n")

            file.write("Metadaten:\n")
            for key, value in doc.metadata.items():
                file.write(f"  - {key}: {value}\n")

            file.write("\nInhalt:\n")
            file.write(f"{doc.page_content}\n\n")

    print(f"Gespeichert: {filepath}")

INFO:root:Nutze für Frage Nr. 1. Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben? den Filter: should=None min_should=None must=[FieldCondition(key='metadata.studiengang', match=MatchValue(value='Elektrotechnik'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.standort', match=MatchValue(value='Meschede'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.abschluss', match=MatchValue(value='Master'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)] must_not=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:root:Nutze für Frage Nr. 2. Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest? den Filter: should=None min_should=None mu

Gespeichert: 3-infloat_token_based-Fragen_mit_Filtern/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt
Gespeichert: 3-infloat_token_based-Fragen_mit_Filtern/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt
Gespeichert: 3-infloat_token_based-Fragen_mit_Filtern/3_Welche Fachgebiete sind an der FH vertreten.txt
Gespeichert: 3-infloat_token_based-Fragen_mit_Filtern/4_Ist eine Beurlaubung während des Studiums möglich.txt
Gespeichert: 3-infloat_token_based-Fragen_mit_Filtern/5_Welche Studienmodelle werden angeboten.txt
Gespeichert: 3-infloat_token_based-Fragen_mit_Filtern/6_Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:root:Nutze für Frage Nr. 8. Was muss ich beachten, wenn ich bei einer Prüfung krank bin? den Filter: should=None min_should=None must=[FieldCondition(key='metadata.studiengang', match=MatchValue(value='Alle'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.standort', match=MatchValue(value='Alle'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.abschluss', match=MatchValue(value='Alle'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)] must_not=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:root:Nutze für Frage Nr. 9. Was sind die Inhalte im Modul IT-

Gespeichert: 3-infloat_token_based-Fragen_mit_Filtern/7_Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut.txt
Gespeichert: 3-infloat_token_based-Fragen_mit_Filtern/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt
Gespeichert: 3-infloat_token_based-Fragen_mit_Filtern/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt
Gespeichert: 3-infloat_token_based-Fragen_mit_Filtern/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt


In [16]:
output_dir = "4-infloat_recursive-Fragen_mit_Filtern"
os.makedirs(output_dir, exist_ok=True)

for id, frage in fragen.items():
    frage_filename = f"{id}_{clean_filename(frage)}.txt"  # Dateiname mit ID ergänzen
    filepath = os.path.join(output_dir, frage_filename)

    logging.info(f"Nutze für Frage Nr. {id}. {frage} den Filter: {filters[id]}")
    found_docs = recursive_client.similarity_search_with_score(frage, k=10, filter=filters[id])

    with open(filepath, "w", encoding="utf-8") as file:
        file.write(f"ID: {id}\n")
        file.write(f"Frage: {frage}\n\n")

        for idx, (doc, score) in enumerate(found_docs, 1):
            file.write(f"Dokument {idx} (Score: {score:.8f}):\n")

            file.write("Metadaten:\n")
            for key, value in doc.metadata.items():
                file.write(f"  - {key}: {value}\n")

            file.write("\nInhalt:\n")
            file.write(f"{doc.page_content}\n\n")

    print(f"Gespeichert: {filepath}")

INFO:root:Nutze für Frage Nr. 1. Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben? den Filter: should=None min_should=None must=[FieldCondition(key='metadata.studiengang', match=MatchValue(value='Elektrotechnik'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.standort', match=MatchValue(value='Meschede'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.abschluss', match=MatchValue(value='Master'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)] must_not=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:root:Nutze für Frage Nr. 2. Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest? den Filter: should=None min_should=None must

Gespeichert: 4-infloat_recursive-Fragen_mit_Filtern/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt
Gespeichert: 4-infloat_recursive-Fragen_mit_Filtern/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt
Gespeichert: 4-infloat_recursive-Fragen_mit_Filtern/3_Welche Fachgebiete sind an der FH vertreten.txt
Gespeichert: 4-infloat_recursive-Fragen_mit_Filtern/4_Ist eine Beurlaubung während des Studiums möglich.txt
Gespeichert: 4-infloat_recursive-Fragen_mit_Filtern/5_Welche Studienmodelle werden angeboten.txt
Gespeichert: 4-infloat_recursive-Fragen_mit_Filtern/6_Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:root:Nutze für Frage Nr. 8. Was muss ich beachten, wenn ich bei einer Prüfung krank bin? den Filter: should=None min_should=None must=[FieldCondition(key='metadata.studiengang', match=MatchValue(value='Alle'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.standort', match=MatchValue(value='Alle'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None), FieldCondition(key='metadata.abschluss', match=MatchValue(value='Alle'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)] must_not=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:root:Nutze für Frage Nr. 9. Was sind die Inhalte im Modul IT-Sich

Gespeichert: 4-infloat_recursive-Fragen_mit_Filtern/7_Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut.txt
Gespeichert: 4-infloat_recursive-Fragen_mit_Filtern/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt
Gespeichert: 4-infloat_recursive-Fragen_mit_Filtern/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt
Gespeichert: 4-infloat_recursive-Fragen_mit_Filtern/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt


## SelfQueryRetriever aus LangChain nutzen:

Dieser Retriever erstellt eigenständig Filterbedingungen anhand der Fragestellung.
Dafür wird ein LLM eingesetzt. Es wird ein Prompt mit Few-Shot-Examples genutzt.
Zusätzlich zu den Few-Shot-Examples werden eigene Schlagworte für das Generieren der Filterbedingungen genutzt.

Zusätzlich ist es möglich, die eingegebene Frage umzuformulieren. Dies soll die Fragestellung vereinfachen und besser kompatibel zu den Datenbankeinträgen machen.

In [17]:
# Importe für LLM und SelfQueryRetriever
from langchain_ollama import ChatOllama
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [18]:
# LLM initialisieren mit ausreichendem Kontextfenster
llm = ChatOllama(
    model="llama3.1:8b", # llama3-2:3b ist nicht ausreichend. Auch mit dem Modell hier kann es noch zu Problemen kommen, die durch leistungsstärkere LLMs gelöst werden könnten (Steht so in der Dokumentation von LangChain).
    temperature=0, # Keine Zufälligkeit
    num_ctx=16000 # Kontextfenster (Das Modell schafft theoretisch 128.000. Aufgrund des erhöhten Speicherbedarfs und meiner Hardware wird hier ein kleineres Kontextfenster festgelegt.)
)

In [19]:
docs_description = "Informationen zum Studium an der Fachhochschule Südwestfalen, den Standorten und Studienmodellen."

metadata_field_info = [
    AttributeInfo(
        name="standort",
        description="Standort des Studiengangs. Einer von ['Alle', 'Iserlohn', 'Hagen', 'Soest', 'Meschede', 'Lüdenscheid']",
        type="string",
    ),
    AttributeInfo(
        name="studiengang",
        description="Der Name des Studiengangs. Einer von ['Alle', 'Wirtschaftsinformatik', 'Elektrotechnik']",
        type="string",
    ),
    AttributeInfo(
        name="abschluss",
        description="Der akademische Grad. Einer von ['Alle', 'Bachelor', 'Master']",
        type="string",
    ),
]

In [20]:
search_kwargs = {"k": 10}
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=token_client,
    document_contents=docs_description,
    metadata_field_info=metadata_field_info,
    use_original_query=True,
    verbose=True,
    search_kwargs=search_kwargs,
)

In [21]:
for frage in fragen.values():
    try:
        result = retriever.invoke(frage)
        print(result)
    except Exception as e:
        print(f"FEHLER: {e}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Master in Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Master'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Meschede')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


[Document(metadata={'id': 1021, 'studiengang': 'Elektrotechnik', 'abschluss': 'Master', 'standort': 'Meschede', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/elektrotechnik_m_eng_/index.php', 'hash': '23d5055fb6686b7f964ad1114b3e41d5d939a5dfe2428fa974f4bd6af87e72a5', '_id': '579aeac7-499f-4213-b1ea-0e9425bb6c64', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='Jahre abhängig von den Studienvoraussetzungen) ### Studienort Meschede ### Beiträge und Gebühren ### Hauptunterrichtssprache Deutsch (für internationale Bewerbende: Level DSH3 / TestDaf5 / C2) ## Vor Beginn des Elektrotechnik Master-Studiums – Voraussetzungen &amp; Bewerbungsverfahren ### Voraussetzungen ##### Für den viersemestrigen Masterstudiengang - Abschluss eines sechssemestrigen Studiums mit mind. 180 Credits in einem der unten aufgeführten Studiengänge mit einer Gesamtnote von mindestens 2,7 (oder mit einer Gesamtnote von mindestens 3,0 und einer Bachelorarbeit mit 

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Mathematik im Bachelorstudiengang Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Soest')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


[Document(metadata={'id': 1254, 'studiengang': 'Elektrotechnik', 'abschluss': 'Bachelor', 'standort': 'Soest', 'link': 'https://www.fh-swf.de/media/neu_np/hv_2/dateien_sg_2_4/verlaufsplaene_modulhandbuecher_1/soest_4/fb_eet_3/Modulhandbuch_BA_ETS_ETA_ETP_20_02.2024.pdf', 'hash': '26302c09bbe8d0ea4045e76eba80ec131257d9aadd1e954f2ac91cdf526af9cb', '_id': '5dd7303a-38f1-49ca-8a55-68539fb88391', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='## Modulhandbuch für die Bachelorstudiengänge Elektrotechnik (B.Eng.) Elektrotechnik dual praxisintegrierend (B.Eng.) Elektrotechnik dual ausbildungsintegrierend (B.Eng.) ## Fachprüfungsordnung 2020 Stand Sommersemester 2024 Fachbereich Elektrische Energietechnik Standort Soest Alle Angaben ohne Gewähr. Verbindlich ist die Fachprüfungsordnung mit Änderungsordnungen in ihren in den Amtlichen Bekanntmachungen der Fachhochschule Südwestfalen veröffentlichten Fassungen. Modulhandbuch Elektrotechnik FPO 2020 Stand 02

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Fachgebiete an der FH' filter=None limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


...

[Document(metadata={'id': 1555, 'studiengang': 'Alle', 'abschluss': 'Alle', 'standort': 'Alle', 'link': 'https://www.fh-swf.de/media/neu_np/hv_2/dateien_sg_2_4/online_pruefungsorga/Ruecktritt-Info-2023.pdf', 'hash': '4bfbc87a928c279860db7fc32f7710789890f3fb6bc4d9451e65003863db6c7e', '_id': '5ce5d003-3ea5-4376-b456-c3ef036542a2', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='sequenzen ziehen und rechtzeitig von der Prüfung zurücktreten. Unterziehen Sie sich der Prüfung, obwohl Sie Ihre Prüfungsunfähigkeit kannten oder den Umständen nach kennen mussten, so handeln Sie auf eigenes Risiko. In diesem Falle können Sie sich weder nach Erbringen der Prüfungsleistung noch beim Abbruch der Prüfung auf Ihre Prüfungsunfähigkeit berufen. Nur für den Fall, dass die Krankheit vor der Prüfung für Sie nicht erkennbar war oder sich während der Prüfung wesentlich verschlimmert hat, kann ggf. eine Berücksichtigung der Prüfungsunfähigkeit bei Prüfungsabbruch unter 

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Modul IT-Sicherheit' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


[Document(metadata={'id': 538, 'studiengang': 'Elektrotechnik', 'abschluss': 'Bachelor', 'standort': 'Hagen', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/elektrotechnik_b_eng_/elektrotechnik_b_eng_1.php', 'hash': '3fa1f100a84b4369bdda4ec5a2d47b43bc02fad9cfc3c169985c0bde7cc162fc', '_id': '04a70a6d-ec3a-4862-be52-84c321cc7a54', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='# Elektrotechnik B.Eng. ## Hagen 1. Studienangebot 2. Studiengänge 3. Hagen 4. Elektrotechnik B.Eng. ## Bachelor-Studium Elektrotechnik ### Ohne Elektrotechnik geht wenig. Sie macht unser Leben heller, bunter, komfortabler, sicherer. Vieles, was die E-Technik möglich macht, ist längst selbstverständlich. Zum Beispiel die zuverlässige Versorgung mit elektrischer Energie, elektrische Beleuchtung, das Smartphone oder die Automatisierung von mühsamen oder gefährlichen Prozessen. Im zulassungsfreien Bachelorstudiengang Elektrotechnik studieren Sie nah an der Praxi

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Bachelorstudiengang Wirtschaftsinformatik in Hagen' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Wirtschaftsinformatik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


[Document(metadata={'id': 1, 'studiengang': 'Wirtschaftsinformatik', 'abschluss': 'Bachelor', 'standort': 'Hagen', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/wirtschaftsinformatik__b_sc__/wirtschaftsinformatik__b_sc__1.php', 'hash': '52d4e379d2abc0535448fc42b8962da89d850e7865654a3ea8a44648e2549a4a', '_id': '2e1ed528-b0aa-4396-8531-1910430076c4', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='# Wirtschaftsinformatik B.Sc. ## Hagen 1. Studienangebot 2. Studiengänge 3. Hagen 4. Wirtschaftsinformatik (B.Sc.) ## Bachelor-Studium Wirtschaftsinformatik ### Interdisziplinarität! Das ist der große Trumpf des Bachelor-Studiengangs Wirtschaftsinformatik. Er kombiniert Inhalte aus den Bereichen Informatik, Betriebswirtschaftslehre (BWL) und Technik zu einer attraktiven Mischung mit besten beruflichen Perspektiven. Auf dem Lehrplan stehen Fächer wie IT-Sicherheit, Business Intelligence oder Web-Programmierung. Dazu gibt es eine fundierte 

In [22]:
retriever.invoke("Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben?")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Master in Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Master'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Meschede')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


[Document(metadata={'id': 1021, 'studiengang': 'Elektrotechnik', 'abschluss': 'Master', 'standort': 'Meschede', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/elektrotechnik_m_eng_/index.php', 'hash': '23d5055fb6686b7f964ad1114b3e41d5d939a5dfe2428fa974f4bd6af87e72a5', '_id': '579aeac7-499f-4213-b1ea-0e9425bb6c64', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='Jahre abhängig von den Studienvoraussetzungen) ### Studienort Meschede ### Beiträge und Gebühren ### Hauptunterrichtssprache Deutsch (für internationale Bewerbende: Level DSH3 / TestDaf5 / C2) ## Vor Beginn des Elektrotechnik Master-Studiums – Voraussetzungen &amp; Bewerbungsverfahren ### Voraussetzungen ##### Für den viersemestrigen Masterstudiengang - Abschluss eines sechssemestrigen Studiums mit mind. 180 Credits in einem der unten aufgeführten Studiengänge mit einer Gesamtnote von mindestens 2,7 (oder mit einer Gesamtnote von mindestens 3,0 und einer Bachelorarbeit mit 

In [23]:
for frage in fragen.values():
    try:
        result = retriever.invoke(frage)
        print(result)
    except Exception as e:
        print(f"FEHLER: {e}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Master in Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Master'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Meschede')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


[Document(metadata={'id': 1021, 'studiengang': 'Elektrotechnik', 'abschluss': 'Master', 'standort': 'Meschede', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/elektrotechnik_m_eng_/index.php', 'hash': '23d5055fb6686b7f964ad1114b3e41d5d939a5dfe2428fa974f4bd6af87e72a5', '_id': '579aeac7-499f-4213-b1ea-0e9425bb6c64', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='Jahre abhängig von den Studienvoraussetzungen) ### Studienort Meschede ### Beiträge und Gebühren ### Hauptunterrichtssprache Deutsch (für internationale Bewerbende: Level DSH3 / TestDaf5 / C2) ## Vor Beginn des Elektrotechnik Master-Studiums – Voraussetzungen &amp; Bewerbungsverfahren ### Voraussetzungen ##### Für den viersemestrigen Masterstudiengang - Abschluss eines sechssemestrigen Studiums mit mind. 180 Credits in einem der unten aufgeführten Studiengänge mit einer Gesamtnote von mindestens 2,7 (oder mit einer Gesamtnote von mindestens 3,0 und einer Bachelorarbeit mit 

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Mathematik im Bachelorstudiengang Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Soest')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


[Document(metadata={'id': 1254, 'studiengang': 'Elektrotechnik', 'abschluss': 'Bachelor', 'standort': 'Soest', 'link': 'https://www.fh-swf.de/media/neu_np/hv_2/dateien_sg_2_4/verlaufsplaene_modulhandbuecher_1/soest_4/fb_eet_3/Modulhandbuch_BA_ETS_ETA_ETP_20_02.2024.pdf', 'hash': '26302c09bbe8d0ea4045e76eba80ec131257d9aadd1e954f2ac91cdf526af9cb', '_id': '5dd7303a-38f1-49ca-8a55-68539fb88391', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='## Modulhandbuch für die Bachelorstudiengänge Elektrotechnik (B.Eng.) Elektrotechnik dual praxisintegrierend (B.Eng.) Elektrotechnik dual ausbildungsintegrierend (B.Eng.) ## Fachprüfungsordnung 2020 Stand Sommersemester 2024 Fachbereich Elektrische Energietechnik Standort Soest Alle Angaben ohne Gewähr. Verbindlich ist die Fachprüfungsordnung mit Änderungsordnungen in ihren in den Amtlichen Bekanntmachungen der Fachhochschule Südwestfalen veröffentlichten Fassungen. Modulhandbuch Elektrotechnik FPO 2020 Stand 02

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Fachgebiete an der FH' filter=None limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


...

[Document(metadata={'id': 1555, 'studiengang': 'Alle', 'abschluss': 'Alle', 'standort': 'Alle', 'link': 'https://www.fh-swf.de/media/neu_np/hv_2/dateien_sg_2_4/online_pruefungsorga/Ruecktritt-Info-2023.pdf', 'hash': '4bfbc87a928c279860db7fc32f7710789890f3fb6bc4d9451e65003863db6c7e', '_id': '5ce5d003-3ea5-4376-b456-c3ef036542a2', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='sequenzen ziehen und rechtzeitig von der Prüfung zurücktreten. Unterziehen Sie sich der Prüfung, obwohl Sie Ihre Prüfungsunfähigkeit kannten oder den Umständen nach kennen mussten, so handeln Sie auf eigenes Risiko. In diesem Falle können Sie sich weder nach Erbringen der Prüfungsleistung noch beim Abbruch der Prüfung auf Ihre Prüfungsunfähigkeit berufen. Nur für den Fall, dass die Krankheit vor der Prüfung für Sie nicht erkennbar war oder sich während der Prüfung wesentlich verschlimmert hat, kann ggf. eine Berücksichtigung der Prüfungsunfähigkeit bei Prüfungsabbruch unter 

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Modul IT-Sicherheit' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


[Document(metadata={'id': 538, 'studiengang': 'Elektrotechnik', 'abschluss': 'Bachelor', 'standort': 'Hagen', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/elektrotechnik_b_eng_/elektrotechnik_b_eng_1.php', 'hash': '3fa1f100a84b4369bdda4ec5a2d47b43bc02fad9cfc3c169985c0bde7cc162fc', '_id': '04a70a6d-ec3a-4862-be52-84c321cc7a54', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='# Elektrotechnik B.Eng. ## Hagen 1. Studienangebot 2. Studiengänge 3. Hagen 4. Elektrotechnik B.Eng. ## Bachelor-Studium Elektrotechnik ### Ohne Elektrotechnik geht wenig. Sie macht unser Leben heller, bunter, komfortabler, sicherer. Vieles, was die E-Technik möglich macht, ist längst selbstverständlich. Zum Beispiel die zuverlässige Versorgung mit elektrischer Energie, elektrische Beleuchtung, das Smartphone oder die Automatisierung von mühsamen oder gefährlichen Prozessen. Im zulassungsfreien Bachelorstudiengang Elektrotechnik studieren Sie nah an der Praxi

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Bachelorstudiengang Wirtschaftsinformatik in Hagen' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Wirtschaftsinformatik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


[Document(metadata={'id': 1, 'studiengang': 'Wirtschaftsinformatik', 'abschluss': 'Bachelor', 'standort': 'Hagen', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/wirtschaftsinformatik__b_sc__/wirtschaftsinformatik__b_sc__1.php', 'hash': '52d4e379d2abc0535448fc42b8962da89d850e7865654a3ea8a44648e2549a4a', '_id': '2e1ed528-b0aa-4396-8531-1910430076c4', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='# Wirtschaftsinformatik B.Sc. ## Hagen 1. Studienangebot 2. Studiengänge 3. Hagen 4. Wirtschaftsinformatik (B.Sc.) ## Bachelor-Studium Wirtschaftsinformatik ### Interdisziplinarität! Das ist der große Trumpf des Bachelor-Studiengangs Wirtschaftsinformatik. Er kombiniert Inhalte aus den Bereichen Informatik, Betriebswirtschaftslehre (BWL) und Technik zu einer attraktiven Mischung mit besten beruflichen Perspektiven. Auf dem Lehrplan stehen Fächer wie IT-Sicherheit, Business Intelligence oder Web-Programmierung. Dazu gibt es eine fundierte 

In [24]:
neuer_test = retriever.invoke("Was muss ich beachten, wenn ich bei einer Prüfung krank bin?")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='krank bei Prüfung' filter=None limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


In [25]:
print(neuer_test)

[Document(metadata={'id': 1555, 'studiengang': 'Alle', 'abschluss': 'Alle', 'standort': 'Alle', 'link': 'https://www.fh-swf.de/media/neu_np/hv_2/dateien_sg_2_4/online_pruefungsorga/Ruecktritt-Info-2023.pdf', 'hash': '4bfbc87a928c279860db7fc32f7710789890f3fb6bc4d9451e65003863db6c7e', '_id': '5ce5d003-3ea5-4376-b456-c3ef036542a2', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='sequenzen ziehen und rechtzeitig von der Prüfung zurücktreten. Unterziehen Sie sich der Prüfung, obwohl Sie Ihre Prüfungsunfähigkeit kannten oder den Umständen nach kennen mussten, so handeln Sie auf eigenes Risiko. In diesem Falle können Sie sich weder nach Erbringen der Prüfungsleistung noch beim Abbruch der Prüfung auf Ihre Prüfungsunfähigkeit berufen. Nur für den Fall, dass die Krankheit vor der Prüfung für Sie nicht erkennbar war oder sich während der Prüfung wesentlich verschlimmert hat, kann ggf. eine Berücksichtigung der Prüfungsunfähigkeit bei Prüfungsabbruch unter 

In [26]:
result = retriever.invoke("Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen?")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Bachelorstudiengang Wirtschaftsinformatik in Hagen' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Wirtschaftsinformatik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


In [27]:
print(result)

[Document(metadata={'id': 1, 'studiengang': 'Wirtschaftsinformatik', 'abschluss': 'Bachelor', 'standort': 'Hagen', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/wirtschaftsinformatik__b_sc__/wirtschaftsinformatik__b_sc__1.php', 'hash': '52d4e379d2abc0535448fc42b8962da89d850e7865654a3ea8a44648e2549a4a', '_id': '2e1ed528-b0aa-4396-8531-1910430076c4', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='# Wirtschaftsinformatik B.Sc. ## Hagen 1. Studienangebot 2. Studiengänge 3. Hagen 4. Wirtschaftsinformatik (B.Sc.) ## Bachelor-Studium Wirtschaftsinformatik ### Interdisziplinarität! Das ist der große Trumpf des Bachelor-Studiengangs Wirtschaftsinformatik. Er kombiniert Inhalte aus den Bereichen Informatik, Betriebswirtschaftslehre (BWL) und Technik zu einer attraktiven Mischung mit besten beruflichen Perspektiven. Auf dem Lehrplan stehen Fächer wie IT-Sicherheit, Business Intelligence oder Web-Programmierung. Dazu gibt es eine fundierte 

In [28]:
output_dir = "5-infloat_token_based-Fragen_mit_SelfQueryRetriever"
os.makedirs(output_dir, exist_ok=True)

for id, frage in fragen.items():
    frage_filename = f"{id}_{clean_filename(frage)}.txt"
    filepath = os.path.join(output_dir, frage_filename)

    try:
        found_docs = retriever.invoke(frage)

        with open(filepath, "w", encoding="utf-8") as file:
            file.write(f"ID: {id}\n")
            file.write(f"Frage: {frage}\n\n")

            for idx, doc in enumerate(found_docs, 1):
                file.write(f"Dokument {idx}:\n")

                file.write("Metadaten:\n")
                for key, value in doc.metadata.items():
                    file.write(f"  - {key}: {value}\n")

                file.write("\nInhalt:\n")
                file.write(f"{doc.page_content}\n\n")

        print(f"Gespeichert: {filepath}")

    except Exception as e:
        print(f"FEHLER: {e}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Master in Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Master'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Meschede')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 5-infloat_token_based-Fragen_mit_SelfQueryRetriever/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Mathematik im Bachelorstudiengang Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Soest')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 5-infloat_token_based-Fragen_mit_SelfQueryRetriever/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Fachgebiete an der FH' filter=None limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


...

Gespeichert: 5-infloat_token_based-Fragen_mit_SelfQueryRetriever/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Modul IT-Sicherheit' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 5-infloat_token_based-Fragen_mit_SelfQueryRetriever/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Bachelorstudiengang Wirtschaftsinformatik in Hagen' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Wirtschaftsinformatik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 5-infloat_token_based-Fragen_mit_SelfQueryRetriever/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt


In [29]:
search_kwargs = {"k": 10}
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=recursive_client,
    document_contents=docs_description,
    metadata_field_info=metadata_field_info,
    use_original_query=True,
    verbose=True,
    search_kwargs=search_kwargs,
)

In [30]:
output_dir = "6-infloat_recursive-Fragen_mit_SelfQueryRetriever"
os.makedirs(output_dir, exist_ok=True)

for id, frage in fragen.items():
    frage_filename = f"{id}_{clean_filename(frage)}.txt"
    filepath = os.path.join(output_dir, frage_filename)

    try:
        found_docs = retriever.invoke(frage)

        with open(filepath, "w", encoding="utf-8") as file:
            file.write(f"ID: {id}\n")
            file.write(f"Frage: {frage}\n\n")

            for idx, doc in enumerate(found_docs, 1):
                file.write(f"Dokument {idx}:\n")

                file.write("Metadaten:\n")
                for key, value in doc.metadata.items():
                    file.write(f"  - {key}: {value}\n")

                file.write("\nInhalt:\n")
                file.write(f"{doc.page_content}\n\n")

        print(f"Gespeichert: {filepath}")

    except Exception as e:
        print(f"FEHLER: {e}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Master in Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Master'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Meschede')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 6-infloat_recursive-Fragen_mit_SelfQueryRetriever/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Mathematik im Bachelorstudiengang Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Soest')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 6-infloat_recursive-Fragen_mit_SelfQueryRetriever/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Fachgebiete an der FH' filter=None limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"


...

Gespeichert: 6-infloat_recursive-Fragen_mit_SelfQueryRetriever/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Modul IT-Sicherheit' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 6-infloat_recursive-Fragen_mit_SelfQueryRetriever/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Bachelorstudiengang Wirtschaftsinformatik in Hagen' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Wirtschaftsinformatik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 6-infloat_recursive-Fragen_mit_SelfQueryRetriever/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt
